# CS446/519 Homework Assignment 2

load in the E. coli operon-operon network (gene regulatory network) data in edge-list format; display the first six lines of data as a sanity check. Use `pandas.read_csv` and `pandas.DataFrame.head`.

In [53]:
import pandas
edge_list_df = pandas.read_csv("shared/ecolitfnet.txt", sep="\t", names=["source","target"])
edge_list_df.shape
edge_list_df.head(n=6)

,source,target
0,ADA,AIDB
1,ADIY,GADX
2,ALLR,ALLS
3,ARCA,BETI
4,ARCA,DPIA
5,ARCA,FNR


Make an igraph directed graph from the network; print a graph summary as a sanity check.  You will need to use `igraph.Graph.TupleList` and `summary`.

In [54]:
import igraph
genereg_graph = igraph.Graph.TupleList(edge_list_df.values.tolist(), directed=True)
genereg_graph.summary()

'IGRAPH DN-- 133 261 -- \n+ attr: name (v)'

Obtain the counts of all three-vertex motifs in the graph. You will need to use the `motifs_randesu` method in `igraph.Graph`.

In [55]:
motif_counts_3 = genereg_graph.motifs_randesu(3)
motif_counts_3

[nan, nan, 275, nan, 125, 39, 1676, 47, 25, 68, 0, 0, 2, 10, 1, 1]

Get motif counts for 10,000 different randomizations of the network.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [50]:
import itertools
motif_counts_rand = []
for _ in itertools.repeat(None, 10000):
    genereg_graph_rand = genereg_graph.copy()
    genereg_graph_rand.rewire(n=1000)
    motif_counts_rand.append(genereg_graph_rand.motifs_randesu(size=3))

For each motif type, compute the mean and standard deviation of the counts of appearances of the motif in each of the 10,000 randomly rewired graphs. You will need to use the `numpy.mean` and `numpy.std` functions.

In [51]:
import numpy
motif_counts_rand_np = numpy.array(motif_counts_rand)
motif_counts_rand_mean = numpy.mean(motif_counts_rand_np, axis=0)
motif_counts_rand_stdev = numpy.std(motif_counts_rand_np, axis=0)

For each motif type, obtain the Z score for the number of appearances of the motif in the real network, vs. in the randomly rewired networks.

In [61]:
motif_counts_zscores = (numpy.array(motif_counts_3) - motif_counts_rand_mean)/motif_counts_rand_stdev

Print out the mean number of FFLs on random networks, the number of FFLs on the real network, and the Z-score for the FFL count on the real network vs. the random networks. The FFL should be in the 7th element (starting from zero!) of the vector returned from `motifs_randesu`.

In [60]:
print("Mean number of FFLs on random networks: ")
print(motif_counts_rand_mean[7])
print("Number of FFLs on the real network: ")
print(motif_counts_3[7])
print("Z-score for the FFL count on the real network vs. the random networks: ")
print(motif_counts_zscores[7])

Mean number of FFLs on random networks: 
57.2775
Number of FFLs on the real network: 
47
Z-score for the FFL count on the real network vs. the random networks: 
-1.11328351937


Figure out the isomorphism class of the network shown in the assignment PDF.  You'll want to build an edge-list tuple-list and then an igraph `Graph` object using `Graph.TupleList` (make sure to specify a directed graph!), and then use the `isoclass` method.

In [49]:
edge_list_4pairs_testnet = [[0,2],[1,2],[0,3],[1,3]]
graph_4pairs_testnet = igraph.Graph.TupleList(edge_list_4pairs_testnet, directed=True)
print(graph_4pairs_testnet.summary())
pairs_graph_isoclass = graph_4pairs_testnet.isoclass()
pairs_graph_isoclass

IGRAPH DN-- 4 4 -- 
+ attr: name (v)


19

Compute the number of appearances of this four-vertex motif in the real network, using `motifs` with `size=4`.

In [58]:
motif_counts_4 = genereg_graph.motifs_randesu(4)
motif_counts_4[pairs_graph_isoclass]

297

Get the mean and standard deviation for the count of four-vertex motifs in 10,000 randomly rewired graphs.  You will need to use a `for` loop, `igraph.Graph.copy`, `igraph.graph.rewire`, and `append`.  You can specify n=1000 for the rewiring.

In [59]:
import itertools
motif_counts_rand4 = []
for _ in itertools.repeat(None, 10000):
    genereg_graph_rand = genereg_graph.copy()
    genereg_graph_rand.rewire(n=1000)
    motif_counts_rand4.append(genereg_graph_rand.motifs_randesu(size=4)[pairs_graph_isoclass])

Print out the mean number of 4-vertex DOR motifs for the random networks, the standard deviation, and the Z score for the DOR motif count in the real graph vs. the random graphs. You'll need to use `numpy.array`, `numpy.mean`, and `numpy.std`.

In [64]:
motif_counts_rand4_np = numpy.array(motif_counts_rand4)
motif_counts_rand4_mean = numpy.mean(motif_counts_rand4)
motif_counts_rand4_std = numpy.std(motif_counts_rand4)
motif_counts_4_Z = (motif_counts_4[pairs_graph_isoclass] - motif_counts_rand4_mean)/motif_counts_rand4_std
print("Average count of the four-vertex DOR motif for random networks: ")
print(motif_counts_rand4_mean)
print("Standard deviation of the count of the four-vertex DOR motif for random networks: ")
print(motif_counts_rand4_std)
print("Z-score for DOR motif count in real vs. random networks: ")
print(motif_counts_4_Z)

Average count of the four-vertex DOR motif for random networks: 
178.1201
Standard deviation of the count of the four-vertex DOR motif for random networks: 
38.6521936763
Z-score for DOR motif count in real vs. random networks: 
3.07563138578
